# 背景
生成モデルでオキシムエステルの生成を行いましたが、数が多いのでうまくフィルタリングしたいです。
しかし、合成しやすいものを渡辺一人でScifinderでチェックして探索するのは大変なので、皆さんのお力を借りたいです。

# 当日やること

## 手順
1. `oxime_with_keton.csv`を読み込む
2. 下の表の条件で化合物を絞り込む
3. 絞れた中で、**合成しやすそうな分子**の**ケトン体**をScfinderで調査する
4. 合成できるものをいくつかピックアップする

## CSVファイルについての説明

| 指標、カラム名 | 絞り込む値 | 指標についての説明 |
| -------------- | ---------------- | -------------- |
| T1 | 2.0eV以上 | 予測したT1エネルギー(eV)です。開裂のエネルギー源（高いほど良い?） |
| S1 | 3.5eV以下 | 予測したS1エネルギー(eV)です。光吸収波長。350 nm~に光吸収があるもの |
| S1_osc | 0.25以上 | 予測したS1振動子強度です。ある程度吸収の強度があって、オキシムエステルの光開裂が進むもの |
| similarity | 0.7以下 | 端的に言うと、生成モデルの学習に使用したデータセットの中の分子との類似度です。1に近いほど似ています。学習に使ったデータと類似していなくて新規性があるものを取り出します。 |
| oxime_num | 1以上 | オキシムエステル活性基の数。 |
| molwt | 300~600 | 分子量。※分子量は合成しやすさの指標としています。もちろん分子量が合成のしやすさとは明確に関連するわけではないので、柔軟に変えて大丈夫です。 |
| sascore | ??? | ケトンの合成難易度です。参考までに（個人的にはあまり信用していないです。）|




In [17]:
import pandas as pd
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit import Chem

In [18]:
df = pd.read_csv("oxime_with_keton.csv", index_col=0)
df.head()

,Smiles,T1,S1,S1_osc,similarity,Scaffold,oxime_num,keton,molwt,SAScore
0,CC(=O)O\N=C/c1ccc(Cl)cc1,2.627373,4.566069,0.879576,1.000000,c1ccccc1,1,NaN,197.621,NaN
1,C/C(=N/OC(=O)c1ccccc1)c1ccccc1,2.830630,4.727700,0.575377,1.000000,O=C(ON=Cc1ccccc1)c1ccccc1,1,CC(=O)c1ccccc1,239.274,1.097109
2,O=C(O/N=C/c1ccccc1)C(=O)ON=C(c1ccccc1)C(c1cccc...,2.487950,4.363856,0.972207,0.536585,O=C(ON=C(c1ccccc1)C(c1ccccc1)c1ccccc1)C(=O)O/N...,2,O=C(c1ccccc1)C(c1ccccc1)c1ccccc1,462.505,1.562575
3,CCCCCC(/C(=N/OC(=O)C)c1ccc(Cl)cc1)=N\OC(=O)C,2.504756,4.149163,0.506938,1.000000,c1ccccc1,2,CCCCCC(=O)C(=O)c1ccc(Cl)cc1,352.818,1.827869
4,C/C(CC(=O)O/N=C\c1ccccc1)=N\OC(C)=O,2.540638,4.326961,0.580047,1.000000,c1ccccc1,2,CC(=O)CC(=O)O/N=C\c1ccccc1,262.265,2.316034
